In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial import Voronoi, voronoi_plot_2d
from scipy.spatial import distance
from sympy import symbols, Eq, solve
import math
import itertools

In [ ]:
# формируем центрыдля построения диаграммы Вороного

points_count = 8
n = points_count ** 4

a = np.array([2, 0, 0, 0], float)
b = np.array([1, 1, 0, 0], float)
c = np.array([1, 0, 1, 0], float)
d = np.array([1, 0, 0, 1], float)

x = np.zeros(n, float)
y = np.zeros(n, float)
z = np.zeros(n, float)
t = np.zeros(n, float)

counter = 0
coords4 = []
print()
for i in range(- (points_count // 4), points_count - (points_count // 4)):
    for j in range(- (points_count // 4), points_count - (points_count // 4)):
        for k in range(- (points_count // 4), points_count - (points_count // 4)):
            for m in range(- (points_count // 4), points_count - (points_count // 4)):
                l = []

                x[counter] = (a * i + b * j + c * k + d * m)[0]
                y[counter] = (a * i + b * j + c * k + d * m)[1]
                z[counter] = (a * i + b * j + c * k + d * m)[2]
                t[counter] = (a * i + b * j + c * k + d * m)[3]

                l.append((a * i + b * j + c * k + d * m)[0])
                l.append((a * i + b * j + c * k + d * m)[1])
                l.append((a * i + b * j + c * k + d * m)[2])
                l.append((a * i + b * j + c * k + d * m)[3])

                coords4.append(l)
                counter += 1
                
coords4 # координаты центров

In [ ]:
# строим диаграмму Вороного

vor4 = Voronoi(coords4)

In [ ]:
# вершины диаграммы Вороного

vor4.vertices

In [ ]:
# регионы

vor4.regions

In [ ]:
# список конечных областей Вороного (-1 в координатах означает, что область бесконечна)

count = 0
vor4.regions_finite = []
for i in vor4.regions:
    if -1 not in i:
        vor4.regions_finite.append(i)
        count += 1
        print(i, len(i))
print(count)

In [ ]:
# максимальный размер региона (по числу вершин)

len_r_max = 24 # размер максимального региона
count_r = 0 # количество таких регионов

for i in range(len(vor4.regions)):
    len_r = len(vor4.regions[i])
    
    if len_r >= len_r_max:
        len_r_max = len_r
        count_r += 1 # считаем количество таких регионов
        
print ('количество вершин =', len_r_max, 'количество =', count_r)

In [ ]:
# находим суммарные расстояния от вершин многогранников до (0, 0, 0, 0) (sum_dist), затем находим минимальное 
# значение. Таким образом находим ближайший центральный многогранник

sum_dist_min = 1000

for i in range(1, len(vor4.regions)):
    len_r = len(vor4.regions[i])

    if -1 not in vor4.regions[i]: # регион должен быть конечным
        l = 0
        
        # для i региона перебираем все его вершины и для каждой ищем расстояние до [0.0, 0.0, 0.0, 0.0]
        for j in range(len_r):
            l += distance.euclidean(vor4.vertices[vor4.regions[i][j]], [0.0, 0.0, 0.0, 0.0])

        if l < sum_dist_min:
            sum_dist_min = l
            v_min = i

        
print ('суммарное расстояние =', sum_dist_min, 'индекс центрального региона =', v_min)

In [ ]:
# центральный регион

central = vor4.vertices[vor4.regions[v_min]]
central

In [ ]:
# индексы вершин центрального региона

vor4.regions[v_min]

In [ ]:
# список многоугольников

vor4.ridge_vertices

In [ ]:
# ищем все многогранники центрального региона

list_polygons = [] # список многогранников центрального региона

# перебираем все многогранники
for i in range(len((vor4.ridge_vertices))):
    count = 0
    
    # перебираем в каждом многограннике все вершины
    for j in range(len((vor4.ridge_vertices[i]))):
    
        # если вершина есть в центральном регионе, то прибавляем к счетчику 1
        if vor4.ridge_vertices[i][j] in vor4.regions[v_min]:
            count += 1
            
    # если все вершины многогранника есть в центральном регионе, то добавляем его в список
    if count == len(vor4.ridge_vertices[i]):
        list_polygons.append(vor4.ridge_vertices[i])
 
            
list_polygons

In [ ]:
# диаметр многогранника (ищем максимальное расстояние между вершинами)

max_len = 0

for i in range(len(central) - 1):
    for j in range(i+1, len(central)):
        len_v = distance.euclidean(central[i], central[j])
        if len_v > max_len:
            max_len = len_v
            u1 = i
            u2 = j
               
print('диаметр =', max_len, 'индексы вершин:', u1, ',',  u2) # u1, u2 - индексы вершин, расстояние между которыми максимально

In [ ]:
# координаты проекции

# coord1, coord2, coord3 - координаты 3 точек, d - точка, проекцию которой надо найти

def coords_proj(coord1, coord2, coord3, d):
    
    M = []        
    M.append(coord1)
    
    l1 = []
    l2 = []
    
    for i in range(4):
        l1.append(coord2[i] - coord1[i])
        l2.append(coord3[i] - coord1[i])
        
    M.append(l1)
    M.append(l2)
    H1 = (np.array(M[1])).dot((np.array(M).T))
    K1 = np.array(l1).dot(d)
    H2 = (np.array(M[2])).dot((np.array(M).T))
    K2 = np.array(l2).dot(d)

    # Определение переменных
    x1, x2 = symbols('x1 x2')

    # Определение системы уравнений
    equations = [
        Eq(H1[0] + H1[1]*x1 + H1[2]*x2, K1),
        Eq(H2[0] + H2[1]*x1 + H2[2]*x2, K2)
    ]

    # Решение системы
    symbolic_solution = solve(equations, (x1, x2))

    # Вывод решения
    symbolic_solution
    
    coords = [1]
    coords.append(symbolic_solution[x1])
    coords.append(symbolic_solution[x2])
    
    
    return np.array(M).T.dot(coords)

In [ ]:
# Проверка - лежит ли точка внутри многоугольника

def area(x, y, z):
    a = math.sqrt((y[0] - x[0]) ** 2 + (y[1] - x[1]) ** 2 + (y[2] - x[2]) ** 2 + (y[3] - x[3]) ** 2)
    b = math.sqrt((y[0] - z[0]) ** 2 + (y[1] - z[1]) ** 2 + (y[2] - z[2]) ** 2 + (y[3] - z[3]) ** 2)
    c = math.sqrt((x[0] - z[0]) ** 2 + (x[1] - z[1]) ** 2 + (x[2] - z[2]) ** 2 + (x[3] - z[3]) ** 2)
    p = (a + b + c) / 2
    res = math.sqrt(p * (p - a) * (p - b) * (p - c))
    return res

def isInside(coord1, coord2, coord3, s):
    count = 0
                
    a = np.array(coord1)
    b = np.array(coord2)
    c = np.array(coord3)

    # Считаем площадь треугольника ABC
    A = area (a, b, c)

    # Считаем площадь треугольника PBC 
    A1 = area (a, b, s)

    # Считаем площадь треугольника PAC 
    A2 = area (a, c, s)

    # Считаем площадь треугольника PAB 
    A3 = area (b, c, s)

    # Проверяем A = A1 + A2 + A3 
    if(A == A1 + A2 + A3):
         return True
        
    else: 
         return False

In [ ]:
# расстояние от точки до отрезка

import math

def distance_point_to_edge(p, a, b):
    def dot_product(v, w):
        sum_coord = 0
        for i in range(len(v)):
            sum_coord += v[i]*w[i]
        return sum_coord

    def vector(v, w):
        list_dif = []
        for i in range(len(v)):
            k = w[i] - v[i]
            list_dif.append(k)
        return list_dif

    def magnitude(v):
        sum_sqr = 0
        for i in range(len(v)):
            sum_sqr += v[i]*v[i]
        return math.sqrt(sum_sqr)

    def scale(v, const):
        list_const = []
        for i in range(len(v)):
            list_const.append(v[i]*const)
        return list_const

    def add(v, w):
        sum_add = []
        for i in range(len(v)):
            sum_add.append(v[i]+w[i])
        return sum_add

    ab = vector(a, b)
    ap = vector(a, p)

    ab_magnitude = magnitude(ab)
    ap_dot_ab = dot_product(ap, ab)

    if ap_dot_ab <= 0:
        
        # Проекция точки на отрезок находится вне отрезка и ближе к точке a
        return magnitude(ap)
    
    if ap_dot_ab >= ab_magnitude*ab_magnitude:
        
        # Проекция точки на отрезок находится вне отрезка и ближе к точке b
        dist_b = 0
        for i in range(len(b)):
            dist_b += ((p[i] - b[i])**2)
        return math.sqrt(dist_b)

    # Проекция точки на отрезок находится внутри отрезка
    projection = add(a, scale(ab, ap_dot_ab/ab_magnitude**2))
    return magnitude(vector(p, projection))

In [ ]:
# список всевозможных строк

digits = range(-2, 3)
pincode_vars = itertools.product(digits, repeat=4)

list_str = []
for var in pincode_vars:

    if var != (0, 0, 0, 0):
        list_str.append(var)
print(len(list_str))

In [ ]:
# решетка в обычном базисе
grid = np.array([[2, 0, 0, 0], [1, 1, 0, 0], [1, 0, 1, 0], [1, 0, 0, 1]])

# ищет для подрешетки хроматическое число и запрещенное расстояние
def ch_number_forb_segm(grid, sub_grid_0):

    # подрешетка в обычном базисе
    sub_grid = ((grid.T).dot(sub_grid_0.T)).T

    # отношение детерминантов подрешетки и решетки (индекс подрешетки)
    chromatic_number = round(np.linalg.det(sub_grid), 0) / round(np.linalg.det(grid), 0)

    # подрешетка в  обычных координатах 
    points_count = 4
    n = points_count ** 4

    a = sub_grid[0]
    b = sub_grid[1]
    c = sub_grid[2]
    d = sub_grid[3]

    x = np.zeros(n, float)
    y = np.zeros(n, float)
    z = np.zeros(n, float)
    t = np.zeros(n, float)


    #строим подрешетку
    counter = 0
    coords4_1 = []

    for i in range(- (points_count // 4), points_count - (points_count // 4)):
        for j in range(- (points_count // 4), points_count - (points_count // 4)):
            for k in range(- (points_count // 4), points_count - (points_count // 4)):
                for m in range(- (points_count // 4), points_count - (points_count // 4)):
                    l = []
                    x[counter] = (a * i + b * j + c * k + d * m)[0]
                    y[counter] = (a * i + b * j + c * k + d * m)[1]
                    z[counter] = (a * i + b * j + c * k + d * m)[2]
                    t[counter] = (a * i + b * j + c * k + d * m)[3]

                    l.append((a * i + b * j + c * k + d * m)[0])
                    l.append((a * i + b * j + c * k + d * m)[1])
                    l.append((a * i + b * j + c * k + d * m)[2])
                    l.append((a * i + b * j + c * k + d * m)[3])

                    coords4_1.append(l)
                    counter += 1

    # ищем координаты, которые есть в решетке и подрешетке
    common_coord = []
    for i in coords4:
        if i in coords4_1:
            common_coord.append(i)

    # расчитывам расстояние от [0.0, 0.0, 0.0, 0.0] до остальных точек и ищем минимальное расстояние
    min_dist = float('inf')
    for i in range(len(common_coord)):
        if common_coord[i] != [0, 0, 0, 0] and distance.euclidean(common_coord[i], [0.0, 0.0, 0.0, 0.0]) \
                                                                                                    < min_dist:
            min_dist = distance.euclidean(common_coord[i], [0.0, 0.0, 0.0, 0.0])
            min_dist_i = i #расстояние от [0.0, 0.0, 0.0, 0.0] до ближайшего другого центра

    # находим точку s
    s = np.array(common_coord[min_dist_i], float) * 0.5


    #  нахожу грани центрального региона
    edge_central = []
    for i in range(len(vor4.ridge_vertices)):
        count = 0

        for j in range(len(vor4.ridge_vertices[i])):
            if vor4.ridge_vertices[i][j] in vor4.regions[v_min]:
                count += 1 

        if count == len(vor4.ridge_vertices[i]):
            edge_central.append(vor4.ridge_vertices[i])



    # перевожу индексы вершин в координаты
    edge_central_coords = []
    for i in range(len(edge_central)):
        r = []
        for j in range(len(edge_central[i])):
            r.append(vor4.vertices[edge_central[i][j]])
        edge_central_coords.append(np.array(r))

    # координаты проекции и расстояние до плоскости через функцию coords_proj

    min_dist_s = float('inf') #минимальное расстояние до точки s 

    for i in range(len(edge_central_coords)):
        for j in range(len(edge_central_coords[i])-2):
            for k in range(j+1, len(edge_central_coords[i])-1):
                for m in range(k+1, len(edge_central_coords[i])):


                    f = distance.euclidean(s, np.array(coords_proj(edge_central_coords[i][j], \
                                                            edge_central_coords[i][k], \
                                                            edge_central_coords[i][m], s), dtype = 'float'))

            
                    
                    if f < min_dist_s:
                        min_coords = coords_proj(edge_central_coords[i][j], \
                                                 edge_central_coords[i][k], \
                                                 edge_central_coords[i][m], s)

                        # если координаты проекции не в многоугольнике, то ищем расстояние до ребра через 
                        # функцию isInside
                        if isInside(edge_central_coords[i][j], \
                                    edge_central_coords[i][k], \
                                    edge_central_coords[i][m], min_coords) == False:


                            # находим минимальное расстояние до ребра через функцию distance_point_to_edge 
                            
                            if distance_point_to_edge(s, edge_central_coords[i][k], edge_central_coords[i][j])\
                                                                    < min_dist_s:
                                min_dist_s = distance_point_to_edge(s, edge_central_coords[i][k], \
                                                                   edge_central_coords[i][j])

                            if distance_point_to_edge(s, edge_central_coords[i][k], edge_central_coords[i][m])\
                                                                    < min_dist_s: 
                                min_dist_s = distance_point_to_edge(s, edge_central_coords[i][k], \
                                                                   edge_central_coords[i][m])
                            if distance_point_to_edge(s, edge_central_coords[i][m], edge_central_coords[i][j])\
                                                                    < min_dist_s: 
                                min_dist_s = distance_point_to_edge(s, edge_central_coords[i][m], \
                                                                   edge_central_coords[i][j])

                        else:
                            min_dist_s = f
                            print(i, 'проекция внутри')
            
            
    return sub_grid_0, abs(chromatic_number), ((min_dist_s * 2)/ max_len)


In [ ]:
# берем за основу матрицу подрешетки:
  #  [[ 0,  0,  0,  0],
   #  [ 0,  1, -2,  1],
    # [ 0, -1, -1,  2],
     #[ 0,  2,  3,  1]]
# и перебираем матрицы, ставя вместо нулей числа: [-3, -2, -1, 0, 1, 2, 3]

mat = np.array([[0, 0, 0, 0], [0, 1, -2, 1], [0, -1, -1, 2], [0, 2, 3, 1]])
list_mat = []
list_ch = []
list_dist = []
list_val = [-3, -2, -1, 0, 1, 2, 3]
for i1 in list_val:
    mat[3][0] = i1
    for i2 in list_val:
        mat[2][0] = i2
        for i3 in list_val:
            mat[1][0] = i3
            for i4 in list_val:
                mat[0][0] = i4
                for i5 in list_val:
                    mat[0][1] = i5
                    for i6 in list_val:
                        mat[0][2] = i6
                        for i7 in list_val:
                            mat[0][3] = i7
                            det_mat = abs(round(np.linalg.det(mat), 0))
                            
                            # если модуль определителя от 25 до 81, то считаем запрещенное расстояние
                            if det_mat >= 25 and det_mat < 81:
                                sub_grid_0 = np.array(mat)
                                sub_grid_good, chromatic_number, forb_dist = ch_number_forb_segm(grid, \
                                                                                                 sub_grid_0)
                                
                                # если запрещенное расстояние больше 1, то записываем матрицу в список 
                                # подходящих матриц
                                if forb_dist > 1:
                                    list_mat.append(sub_grid_good)
                                    list_ch.append(chromatic_number)
                                    list_dist.append(forb_dist)
                                    print(sub_grid_good, chromatic_number, forb_dist)